In [ ]:
# Notebook: Feature Engineering - desc - 1
# Author: Thomas Purk
# Date: 2025-03-17
# Reference: https://www.kaggle.com/datasets/mchirico/montcoalert

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emergency-911-calls-mcpa/911.csv


## desc - Data Exploration

In [3]:
# Notebook Step up steps

import warnings
warnings.filterwarnings('ignore')

df_in_path = '/kaggle/input/emergency-911-calls-mcpa/911.csv'
df_out_path = '/kaggle/working/911.csv'

# Load the data 
df_911 = pd.read_csv(df_in_path)

def report_null_empty(df, feature):
    ''' Prints the count and percent of total of null or empty feature (column) values.
    
    Parameters: 
        df (dataframe): A Pandas dataframe which contains the fature.
        feature (string): The name of the feature in the dataframe to report on.
    '''
    null_count = df[feature].isnull().sum()
    empty_count = (df[feature] == "").sum()
    false_count = (df[feature] == False).sum()
    nan_count = (df[feature].isna()).sum()
    print('')
    print(f'{feature}: Null / Empty Report')
    print(f'\tRow count: {len(df)}')
    print(f'\tNull count: {null_count}')
    print(f'\tNull percent: {round(null_count / len(df) * 100,6)}%')
    print(f'\tEmpty count: {empty_count}')
    print(f'\tEmpty precent: {round(empty_count / len(df) * 100,6)}%')
    print(f'\tFalse count: {false_count}')
    print(f'\tFalse precent: {round(false_count / len(df) * 100,6)}%')
    print(f'\tNAN count: {nan_count}')
    print(f'\tNAN precent: {round(nan_count / len(df) * 100,6)}%')

def get_one_offs(df, feature):

    # Count occurrences of each value in feature
    value_counts = df[feature].value_counts()
    
    # Identify values that appear only once
    unique_values = value_counts[value_counts == 1].index
    
    # Filter the DataFrame to include only rows with unique values in feature
    return df[df[feature].isin(unique_values)]

In [5]:
# Display basic information
display(df_911.info())
display(df_911.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649696 entries, 0 to 649695
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   desc       649696 non-null  object 
 1   zip        574587 non-null  float64
 2   title      649696 non-null  object 
 3   timeStamp  649696 non-null  object 
 4   twp        649696 non-null  object 
 5   addr       649696 non-null  object 
 6   e          649696 non-null  int64  
 7   twp_type   649696 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 39.7+ MB


None

,desc,zip,title,timeStamp,twp,addr,e,twp_type
0,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER TOWNSHIP,REINDEER CT & DEAD END,1,township
1,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,township
2,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN BOROUGH,HAWS AVE,1,borough
3,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN BOROUGH,AIRY ST & SWEDE ST,1,borough
4,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE TOWNSHIP,CHERRYWOOD CT & DEAD END,1,township


In [6]:
# Inspect the desc feature
print("### desc ###")
display(df_911['desc'].describe())
report_null_empty(df_911,'desc')

### desc ###


count                                                649696
unique                                               649457
top       CITY AVE & CARDINAL AVE;  LOWER MERION; Statio...
freq                                                      5
Name: desc, dtype: object


desc: Null / Empty Report
	Row count: 649696
	Null count: 0
	Null percent: 0.0%
	Empty count: 0
	Empty precent: 0.0%
	False count: 0
	False precent: 0.0%
	NAN count: 0
	NAN precent: 0.0%


In [7]:
# NOTE: desc column values have no nulls and are highly unique
# NOTE: desc column values seem to contain multiple features delimited by ";"
# NOTE: could be a station feature embedded in the desc.

# Inspect split of desc feature
desc_split = df_911['desc'].str.split(';', expand=True)
display(desc_split.info())
display(desc_split.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649696 entries, 0 to 649695
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       649696 non-null  object
 1   1       649696 non-null  object
 2   2       649696 non-null  object
 3   3       649696 non-null  object
 4   4       325044 non-null  object
dtypes: object(5)
memory usage: 24.8+ MB


None

,0,1,2,3,4
0,REINDEER CT & DEAD END,NEW HANOVER,Station 332,2015-12-10 @ 17:10:52,
1,BRIAR PATH & WHITEMARSH LN,HATFIELD TOWNSHIP,Station 345,2015-12-10 @ 17:29:21,
2,HAWS AVE,NORRISTOWN,2015-12-10 @ 14:39:21-Station:STA27,,None
3,AIRY ST & SWEDE ST,NORRISTOWN,Station 308A,2015-12-10 @ 16:47:36,
4,CHERRYWOOD CT & DEAD END,LOWER POTTSGROVE,Station 329,2015-12-10 @ 16:56:52,


## Accumulated Notes
- desc column values have no nulls and are highly unique
- desc column values seem to contain multiple features delimited by ";"
- but ";" is not a reliable delimiter
- could be a station feature embedded in the desc.


**Actions**
- engineer a new 'station' feature by extracting unique text from 'desc'

In [60]:
''' Engineer new feature "station"
    The goal is to find valuable data in the desc field if any
    
    - Clean up "desc" to create a new "station" feature
    - Remove substrings that are repeated in other fields
    - ";" is not a reliable delimiter
'''

# Loop through rows using iterrows()
for index, row in df_911.iterrows():

    # Create a variable containing the string to clean up
    new_value = row['desc']

    # Create a list of clean values to inspect
    # The Desc feature contains timestamps Example: "2015-12-10 @ 17:10:52"
    # But the timeStamp feature is formated as  "2015-12-10 17:10:52"
    # So split the timeStamp by ' @ ' and remove each part seperately
    new_value = new_value.replace(' @ ',' ')
    
    ## The Desc feature contains timestamps Example: "2015-12-10 @ 17:10:52"
    ## But the timeStamp feature is formated as  "2015-12-10 17:10:52"
    ## So split the timeStamp by ' ' and remove each part seperately
    ts_parts = row['timeStamp'].split(' ')
    new_value = new_value.replace(ts_parts[0], '')
    new_value = new_value.replace(ts_parts[1], '')

    
    # Remove addr string
    # Have some bad addr values that are just integers, can't use them because they
    if(not row['addr'].isdigit()):
        new_value = new_value.replace(row['addr'], '')


    # Remove twp string
    # Must account for the normilization of the twp feature that took place previously
    twp = str(row['twp']).replace(' TOWNSHIP','').replace(' BOROUGH','')
    new_value = new_value.replace(twp, '')
    
    # # 1. Raw String
    # new_value = new_value.replace(str(row['twp']), '')
    # # 2. Raw String - ' TOWNSHIP'
    # new_value = new_value.replace(str(row['twp']).replace(' TOWNSHIP',''), '')
    # # 3. Raw String - ' BORO'
    # new_value = new_value.replace(str(row['twp']).replace(' BOROUGH',' BORO'), '')
    # # 4. Raw String - ' BOROUGH'
    # new_value = new_value.replace(str(row['twp']).replace(' BOROUGH',''), '')
  
    # clean up delimiters
    new_value = new_value.replace(';','')
    new_value = new_value.replace('@','')
    new_value = new_value.replace('-','')
    new_value = new_value.replace(':','')

    # "Station" and "STA" are not needed
    new_value = new_value.replace('Station','')
    new_value = new_value.replace('STA','')

    # Must account for the normilization of the twp feature that took place previously
    # so, will need to try 4 version of the string
    new_value = new_value.replace('TOWNSHIP', '').replace('BORO', '').replace('BOROUGH', '')

    # Final white space cleanup
    new_value = new_value.strip()
    
    # Write the remaining string as a new feature
    df_911.at[index, 'station'] = new_value

In [61]:
# Inspect the desc feature
print("### station ###")
display(df_911['station'].describe())
report_null_empty(df_911,'station')

### station ###


count     649696
unique       158
top             
freq      225660
Name: station, dtype: object


station: Null / Empty Report
	Row count: 649696
	Null count: 0
	Null percent: 0.0%
	Empty count: 225660
	Empty precent: 34.733168%
	False count: 0
	False precent: 0.0%
	NAN count: 0
	NAN precent: 0.0%


In [66]:
# 'EMS' & 'FIRE' are too generic for station ids, how many are there
df_911[df_911['station'].isin(['FIRE','EMS'])]['station'].value_counts()

station
EMS     176
FIRE      6
Name: count, dtype: int64

In [67]:
# Set EMS and FIRE to ''
df_911.loc[df_911['station'].isin(['FIRE','EMS']),'station'] = ''

In [71]:
# Look at stations with only one occurance
station_one_offs = get_one_offs(df_911, 'station')
print(f'Station One Off Count: {len(station_one_offs)}')

Station One Off Count: 21


In [69]:
# List unique stations
print(df_911['station'].value_counts().to_string())

station
                          225842
308A                       25321
329                        21879
313                        20309
381                        15937
345                        14468
308                        13947
351                        12695
345B                       12156
317                        11752
382                        11160
322A                       10614
308B                        9126
339                         8032
344                         7766
318                         7761
384                         7717
358                         7608
352                         7431
358A                        6704
385                         6429
324                         6404
322                         6170
345A                        6072
369                         5935
325                         5457
313A                        5391
27                          5256
332                         5235
355                         5194
38

## Additional Notes
- Station has a high percentage of missing values
- But low instance of one-off values could means that it is a decent categorical value
- Based on additional research here https://wiki.radioreference.com/index.php/Montgomery_County_(PA)
- Many of the station ids appear valid.
- Station id's seem to be tied to a dispatch region
- However, station ids may repeat among EMS, Fire, HazMAt, and several type of law enforcement.
- So while it might be possible to validate and match the station id to the correct department via the "title" column
- ... and it might be possble to fill in the 34% missing values via imputation based on the addr column
- ... the quality of the data would be questionable and it is not clear how station would benefit a model analysis
- ... over the addr column
- Decision is to remove both the desc and station columns.

In [72]:
# drop the desc column
df_911.drop('desc', axis=1, inplace=True)

# drop the station column
df_911.drop('station', axis=1, inplace=True)

In [73]:
# Update the file
# After Updating
# 1. Manually Download locally
# 2. Manually Updload to a new version of the Kaggle Dataset


# Check if file exists
if os.path.exists(df_out_path):
    os.remove(df_out_path)
    print(f"File '{df_out_path}' has been deleted.")
else:
    print(f"The file '{df_out_path}' does not exist.")

df_911.to_csv(df_out_path, index=False)

The file '/kaggle/working/911.csv' does not exist.
